# Worksheet 08

Name: Youxuan Ma

UID: U23330522

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [41]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.304541683786297, 5.7690603350501615, 4.713737382068444, 3.8355802350023573, 3.538556623979286, 4.532538270478928, 5.2431143608773345, 6.205365331578056, 4.80581993462951, 4.777304664122473]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [42]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.616520655072516, 8.27167053719687, 9.49210031267588, 7.727777189157077, 8.091439530657986, 7.1395207186394956, 6.546647025697315, 8.234952780475739, 9.51539986467853, 8.932837127756494]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [43]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.777304664122473, 8.932837127756494, 9.51539986467853, 8.234952780475739, 6.546647025697315, 7.1395207186394956, 8.091439530657986, 4.80581993462951, 6.205365331578056, 7.727777189157077]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Parameters of the GMM:

- **Number of Mixture Components:** 2
- **Mixture Weights:** $\pi_1 = 0.5$, $\pi_2 = 0.5$
- **Means:** $\mu_1 = 8$, $\mu_2 = 5$
- **Variances:** $\sigma^2_1 = 1$, $\sigma^2_2 = 1$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [44]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [np.var(s1, ddof=0), np.var(s2, ddof=0)]
# var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

# yes

[8.932837127756494, 9.51539986467853, 8.234952780475739, 7.1395207186394956, 8.091439530657986, 7.727777189157077]
[4.777304664122473, 6.546647025697315, 4.80581993462951, 6.205365331578056]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.273654535227553,  mean_2 = 5.583784239006839
var_1 = 0.5992324796831981,  var_2 = 0.6422763910565864


The parameter estimates given by this process are fairly close to the true values overall, with only var_1 estimate being much lower than its true value.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [45]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # pdf_i[0] is P(X_i | S_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x)
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.777304664122473
probability of observing that point if it came from cluster 0 =  1.9154254449242817e-05
probability of observing that point if it came from cluster 1 =  0.3000213691885735
point =  8.932837127756494
probability of observing that point if it came from cluster 0 =  0.3586347156002098
probability of observing that point if it came from cluster 1 =  8.03487047062314e-05
point =  9.51539986467853
probability of observing that point if it came from cluster 0 =  0.14234887012900982
probability of observing that point if it came from cluster 1 =  2.957932350095597e-06
point =  8.234952780475739
probability of observing that point if it came from cluster 0 =  0.5147183101560714
probability of observing that point if it came from cluster 1 =  0.0020927509289660204
point =  6.546647025697315
probability of observing that point if it came from cluster 0 =  0.04278654319715471
probability of observing that point if it came from cluster 1 =  0.2418823727038795
point =  7.1

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [46]:
prob_s = [sum(prob_s0_x)/len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)])/sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)])/sum(prob_s1_x)]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)])/sum(prob_s0_x) , sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)])/sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6012859758483273,  P(S_2) = 0.3987140241516728
mean_1 = 8.23827591135901,  mean_2 = 5.6284617799556305
var_1 = 0.6920694571889405,  var_2 = 0.762217526317313


Based on the results, the values are almost the same as the original ones got from K means, except the variances are larger and closer to the true values.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [47]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # pdf_i[0] is P(X_i | S_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x)
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.777304664122473
probability of observing that point if it came from cluster 0 =  8.364805709225003e-05
probability of observing that point if it came from cluster 1 =  0.2841041991996034
point =  8.932837127756494
probability of observing that point if it came from cluster 0 =  0.33843100660633313
probability of observing that point if it came from cluster 1 =  0.0003541597737932652
point =  9.51539986467853
probability of observing that point if it came from cluster 0 =  0.14759445592860257
probability of observing that point if it came from cluster 1 =  2.2682377019105992e-05
point =  8.234952780475739
probability of observing that point if it came from cluster 0 =  0.4795476413798386
probability of observing that point if it came from cluster 1 =  0.005301446210960466
point =  6.546647025697315
probability of observing that point if it came from cluster 0 =  0.06066838304767969
probability of observing that point if it came from cluster 1 =  0.26283976046940927
point =  7

There aren't much differences. Maybe this is because our data sample is too small, with only 10 points.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [48]:
updated_probs = zip(prob_s0_x, prob_s1_x)

def hard_assignment(updated_probs):
    
    cluster_assignments = []
    
    for probs in updated_probs:
        cluster_label = np.argmax(probs)
        cluster_assignments.append(cluster_label)
    
    return np.array(cluster_assignments)

cluster_labels = hard_assignment(updated_probs)

print("Cluster assignments for each data point:")
for i, label in enumerate(cluster_labels):
    print(f"Data point {data[i]}: Cluster {label}")


Cluster assignments for each data point:
Data point 4.777304664122473: Cluster 1
Data point 8.932837127756494: Cluster 0
Data point 9.51539986467853: Cluster 0
Data point 8.234952780475739: Cluster 0
Data point 6.546647025697315: Cluster 1
Data point 7.1395207186394956: Cluster 0
Data point 8.091439530657986: Cluster 0
Data point 4.80581993462951: Cluster 1
Data point 6.205365331578056: Cluster 1
Data point 7.727777189157077: Cluster 0
